## Лаба 4. Прогнозирование пола и возрастной категории — Spark Streaming

[git laba4](https://github.com/newprolab/sber-spark-ds-10/blob/main/labs/lab04.md)

У вас имеются данные: логи посещения пользователей по разным сайтам рунета. 

По этим пользователям вам также известны: пол и возрастная категория. 

Вам нужно будет в real-time спрогнозировать эти характеристики по пользователям, о которых у вас нет этой информации, но будут все те же самые логи посещения.

**Описание данных**

Обучающая выборка, с которой вы будете работать, выглядит следующим образом:


Она расположена на hdfs: `/labs/slaba04/`.

Поле `gender` принимает значения F (женщина) и M (мужчина).

Поле `age` принимает значения диапазона возраста: 18-24, 25-34, 35-44, 45-54, >=55

Поле `uid` принимает значения уникального ID пользователя (cookies): d50192e5-c44e-4ae8-ae7a-7cfe67c8b777.

Поле user_json имеет внутри json со следующей схемой данных:

{"visits": [{"url": "url1", "timestamp": "timestamp1"}, {"url": "url2", "timestamp": "timestamp2"}]}. 

В нем содержатся непосредственно логи посещения пользователем страниц вместе с временной меткой посещения.

То есть все то же самое, только без поля gender и age и немного с другой схемой данных. Название топика, в который мы будем присылать данные: input_ivan.ivanov, где вместо ivan.ivanov ваш логин от личного кабинета. Данные вам будут поступать по нажатию кнопки "Проверить" в личном кабинете. Вам топик создавать не нужно. Он будет автоматом создаваться при записи. Чистить его тоже не надо, потому что Spark Streaming хранит оффсеты, то есть знает, что он уже считывал, что еще не считывал.

Порт брокера Kafka: 6667. Hostname: spark-master-1.newprolab.com или spark-node-1.newprolab.com.

**Результат**
Вам в свою очередь нужно будет, считывая данные из Kafka, делать прогноз по ним в Spark Streaming и отправлять в real-time эти прогнозные значения в свой другой топик в Kafka в формате:

{"uid": "fe1dba8f-3131-439f-9031-851c0da0f126", "gender": "M", "age": "25-34"}

{"uid": "d50192e5-c44e-4ae8-ae7a-7cfe67c8b777", "gender": "F", "age": "18-24"}

Название топика — ваш логин в личном кабинете: `ivan.ivanov`. 

После отправки данных чекер будет ждать 90 секунд, чтобы вы обработали входные данные из топика input_ivan.ivanov и записали результаты прогноза уже в этот топик.

In [ ]:
import os
import sys
import re
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 --executor-memory 3g --driver-memory 2g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, ArrayType, BinaryType
from pyspark.sql.types import ByteType
from pyspark.ml.feature import CountVectorizer

import json
from pyspark.ml.linalg import DenseVector, SparseVector
from pyspark.ml.linalg import Vectors
import pyspark.sql.functions as F
from pyspark.sql.window import Window

from pyspark.ml.feature import  VectorAssembler
from pyspark.ml import Pipeline

from pyspark.ml.classification import GBTClassifier

conf = SparkConf()
conf.set("spark.app.name", "Kurseev Maxim Spark 4 lab") 

spark = SparkSession.builder.config(conf=conf).appName("Kurseev Maxim Spark 4 lab").getOrCreate()

In [ ]:
import pyspark.sql.types as T
from pyspark.ml.feature import HashingTF, Tokenizer, StringIndexer, IndexToString
from pyspark.ml.classification import RandomForestClassifier

In [ ]:
!hdfs dfs -cat /labs/slaba04/gender_age_dataset.txt | head -n2

In [ ]:
# формируем spark DF
path = '/labs/slaba04/gender_age_dataset.txt'

schema = t.StructType(fields=[
    t.StructField('gender', t.StringType()),
    t.StructField('age', t.StringType()),
    t.StructField('uid', t.StringType()),
    t.StructField('user_json', t.StringType()),
])

train_data = spark.read.csv(path, header=True, schema=schema, sep='\t')

In [ ]:
# схема для json с визитами
visits_schema = t.StructType([
    t.StructField('visits', t.ArrayType(
        t.StructType([
            t.StructField('url', t.StringType(), True),
            t.StructField('timestamp', t.LongType(), True)
        ])
    ))
])

spark_df_flattened = train_data.withColumn("visits", f.from_json(f.col("user_json"), visits_schema))
spark_df_flattened= spark_df_flattened.withColumn("visit", f.explode("visits.visits").alias("visit"))

In [ ]:
spark_df_flattened = ( spark_df_flattened 
    .withColumn("site", f.expr("parse_url(visit.url, 'HOST')")) 
    .drop("visits", "visit","user_json")
                     )

In [ ]:
train_final = ( spark_df_flattened 
    .groupBy("gender", "age", "uid") 
    .agg(f.collect_list("site") 
    .alias("sites"))
              )

train_final = train_final.na.drop()
train_final.select('uid').distinct().count()

### fitting

In [ ]:
X_train, X_test = train_final.randomSplit([0.75, 0.25], seed=123)

In [ ]:
hashing_TF = HashingTF(
    inputCol='sites', 
    outputCol='features', 
    numFeatures=10000, 
    binary=False)


indexer_age = StringIndexer(
    inputCol='age', 
    outputCol='ageIndex').fit(train_final)
indexer_gender = StringIndexer(
    inputCol='gender', 
    outputCol='genderIndex').fit(train_final)


rf_age = RandomForestClassifier(
    featuresCol = 'features', 
    labelCol = 'ageIndex', 
    predictionCol='age_pred', 
    rawPredictionCol='age_raw_pred',
    probabilityCol = 'age_probab')


rf_gender = RandomForestClassifier(
    featuresCol = 'features', 
    labelCol = 'genderIndex',
    predictionCol='gender_pred', 
    rawPredictionCol='gender_raw_pred',
    probabilityCol = 'gender_prob')


converter_age = IndexToString(
    inputCol='age_pred', 
    outputCol='PredictedAge', 
    labels=indexer_age.labels)

converter_gender = IndexToString(
    inputCol='gender_pred', 
    outputCol='PredictedGender', 
    labels=indexer_gender.labels)

In [ ]:
pipeline = Pipeline(
    stages=[hashing_TF, indexer_age, indexer_gender, rf_age, rf_gender, converter_age, converter_gender])

model = pipeline.fit(train_final)
predictions = model.transform(X_test)

In [ ]:
# calculate accuracy for age
evaluator = MulticlassClassificationEvaluator(labelCol="ageIndex", predictionCol="age_pred")
evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})

# calculate accuracy for age
evaluator = MulticlassClassificationEvaluator(labelCol="genderIndex", predictionCol="gender_pred")
evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})

### streaming

In [ ]:
KAFKA_BOOTSTRAP_SERVER = 'spark-node-2.newprolab.com:6667'
KAFKA_INPUT_TOPIC = 'input_maxim.kurseev'
KAFKA_OUTPUT_TOPIC = 'maxim.kurseev'

In [ ]:
# чтение в статическом режиме

kafka_read_df = (
    spark.read
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('subscribe', KAFKA_INPUT_TOPIC)
    .option('startingOffsets', 'earliest')
    .option('failOnDataLoss', 'False')
    .load()
    .cache()
)

In [ ]:
# Парсинг бинарного файла из кафки

event_schema = t.StructType([
    t.StructField('uid', t.StringType(), True),
    t.StructField('visits', t.StringType(), True),
])


visit_schema = t.ArrayType(
    t.StructType([
        t.StructField('url', t.StringType(), True),
        t.StructField('timestamp', t.LongType(), True)
    ])
)


clean_df = (
    kafka_read_df
    .select(f.col('value').cast('string').alias('value'))
    .select(f.from_json(f.col('value'), event_schema).alias('event'))
    .select(
        'event.uid', 
        f.from_json(f.col('event.visits'), visit_schema).alias('visits')
    )
)


In [ ]:
# извлечение url
# применение модели, сохранение предсказаний в predictions_df  

prep_df = ( clean_df 
    .withColumn("visit", f.explode("visits").alias("visit")) 
    .withColumn("site", f.expr("parse_url(visit.url, 'HOST')")) 
    .drop("visits", "visit") 
    .groupBy("uid") 
    .agg(f.collect_list("site").alias("sites"))
          )

predictions_df = ( model.transform(prep_df) 
    .select("uid", "PredictedGender", "PredictedAge") 
    .withColumnRenamed("PredictedAge","age") 
    .withColumnRenamed("PredictedGender","gender")
                 )

In [ ]:
# Оборачивание предсказания обратно в json

kafka_out_df = (
    predictions_df 
    .select(f.to_json(f.struct(*predictions_df.columns)).alias('value')).limit(5)
)

# Запись в выходной топик

(
    kafka_out_df
    .write
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('topic', KAFKA_OUTPUT_TOPIC)
    .save()
)

In [ ]:
# чтение стрима

kafka_stream = (
    spark
    .readStream
    .format('kafka')
    .option('kafka.bootstrap.servers', KAFKA_BOOTSTRAP_SERVER)
    .option('subscribe', KAFKA_INPUT_TOPIC)
    .option('startingOffsets', 'earliest')
    .option('failOnDataLoss', 'False')
    .load()
)

In [ ]:
# предсказание и запись

clean_df = (
    kafka_stream
    .select(f.col('value').cast('string').alias('value'))
    .select(f.from_json(f.col('value'), event_schema).alias('event'))
    .select(
        'event.uid', 
        f.from_json(f.col('event.visits'), visit_schema).alias('visits')
    )
)

prep_df = ( clean_df 
    .withColumn("visit", f.explode("visits").alias("visit")) 
    .withColumn("site", f.expr("parse_url(visit.url, 'HOST')")) 
    .drop("visits", "visit") 
    .groupBy("uid") 
    .agg(f.collect_list("site").alias("sites"))
          )

predictions_df = ( model.transform(prep_df) 
    .select("uid", "PredictedGender", "PredictedAge") 
    .withColumnRenamed("PredictedAge","age") 
    .withColumnRenamed("PredictedGender","gender")
                 )

kafka_write_stream = (
    predictions_df
    .select(f.to_json(f.struct(*predictions_df.columns)).alias('value'))
    .writeStream
    .format("kafka")
    .option("checkpointLocation", "checkpoints/checkpoints_lab04")
    .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER)
    .option("topic", KAFKA_OUTPUT_TOPIC)
    .outputMode("complete")
    .start()
)

In [ ]:
streams = SparkSession.builder.getOrCreate().streams.active
len(streams), streams[0].lastProgress["sources"][0]["description"]

In [ ]:
kafka_write_stream.isActive

___
### Stop session

In [ ]:
kafka_write_stream.stop()

In [ ]:
spark.stop()